In [ ]:
!pip install h5py
!pip install pandas
!pip install tsfresh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [ ]:
import h5py
import pandas as pd
from google.colab import drive
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import EfficientFCParameters
import multiprocessing
import os
import re

In [ ]:

with h5py.File("sample_data/train.h5", "r") as f:
    # Explorar las claves disponibles
    print("Keys (datasets disponibles):", list(f.keys()))
    # Cargar datos de 'x' y 'y'
    x = f['x'][:]
    y = f['y'][:]

    print("Shape de 'x':", x.shape) # 1 representa el canal por el caul se tomo
    print("Tipos de datos de 'x': ", x.dtype)
    print("Shape de 'y':", y.shape)
    print("Tipos de datos de 'y': ", y.dtype)


Keys (datasets disponibles): ['x', 'y']
Shape de 'x': (204, 1, 18530)
Tipos de datos de 'x':  float64
Shape de 'y': (204,)
Tipos de datos de 'y':  int64


In [ ]:
with h5py.File("sample_data/train.h5", "r") as f_pd:
    x_data = f_pd['x'][:]
    y_data = f_pd['y'][:]
x_df = pd.DataFrame(x_data[:,0,:])
y_df = pd.Series(y_data)

print("Shape de 'x':", x_df.shape)
print("Shape de 'y':", y_df.shape)

# Ver las primeras filas del DataFrame
x_df.head()


Shape de 'x': (204, 18530)
Shape de 'y': (204,)


,0,1,2,3,4,5,6,7,8,9,...,18520,18521,18522,18523,18524,18525,18526,18527,18528,18529
0,-0.005859,-0.011414,-0.010773,-0.010559,-0.008850,-0.008636,-0.009094,-0.008850,-0.009369,-0.009888,...,0.012573,0.007721,0.002075,-0.003174,-0.007843,-0.012573,-0.017670,-0.019897,-0.021637,-0.018860
1,0.002380,0.005127,0.005737,0.007263,0.008575,0.010742,0.012970,0.017212,0.022003,0.027954,...,0.003815,0.005707,0.006683,0.006012,0.003601,0.000427,-0.001984,-0.003998,-0.005981,-0.007660
2,0.058075,0.122040,0.108307,0.113769,0.105316,0.103912,0.094849,0.091187,0.080170,0.074432,...,-0.009613,-0.008301,-0.005402,-0.007690,-0.006592,-0.007904,-0.009674,-0.013153,-0.014130,-0.018250
3,-0.007385,-0.011932,-0.004700,-0.003143,0.005493,0.006744,0.012085,0.012909,0.018097,0.021606,...,-0.003082,-0.007050,-0.010956,-0.013916,-0.017944,-0.019867,-0.025604,-0.027161,-0.030273,-0.039368
4,-0.000458,-0.001099,0.003723,0.002930,0.004059,0.003906,0.003265,0.002777,0.000488,-0.001221,...,0.040039,0.040680,0.041901,0.040710,0.038727,0.025818,0.019470,0.010010,0.004242,0.003235


In [ ]:
y_df.head()

,0
0,0
1,1
2,0
3,0
4,0


In [ ]:
# Parseamos el dataset para poder usar la libreía de las series temporales
def parsear_time_series(dataset):
    parsed = []
    for id, series in enumerate(dataset.values):
        # Aplanar la serie (la serie tiene forma (1, N), la aplanamos)
        series = series.flatten()

        # Crear el DataFrame para cada serie temporal
        df = pd.DataFrame({
            'id': id,  # Identificador de la serie
            'time': range(len(series)),  # Tiempo
            'valor': series  # Los valores de la serie temporal
        })
        parsed.append(df)

    # Concatenamos todos los DataFrames en uno solo
    result = pd.concat(parsed, ignore_index=True)

    return result



In [ ]:
df_ready = parsear_time_series(x_df)

print("EL shape de df_rady para la extracción es: ", df_ready.shape)
df_ready.head()


EL shape de df_rady para la extracción es:  (3780120, 3)


,id,time,valor
0,0,0,-0.005859
1,0,1,-0.011414
2,0,2,-0.010773
3,0,3,-0.010559
4,0,4,-0.008850


In [ ]:
import multiprocessing
print("Número de CPUs disponibles:", multiprocessing.cpu_count())





Número de CPUs disponibles: 2


In [ ]:


# Montamos Google Drive
drive.mount('/content/drive')

# Definimos la ruta de salida
output_dir = "/content/drive/MyDrive/features_blocks"
os.makedirs(output_dir, exist_ok=True)

# Iterar sobre cada ID único en el DataFrame
unique_ids = df_ready["id"].unique()

# Procesar cada ID individualmente
for id_value in unique_ids:
    print(f"Procesando ID: {id_value}...")

    # Filtrar el DataFrame para solo ese ID
    df_id = df_ready[df_ready["id"] == id_value]

    # Extraer características para ese ID
    features_id = extract_features(df_id,
                                   column_id="id",
                                   column_sort="time",
                                   default_fc_parameters=EfficientFCParameters(),
                                   disable_progressbar=False,
                                   n_jobs=multiprocessing.cpu_count())
    # Rellenamos con 0 los NaN
    features_id = features_id.fillna(0)
    print("EL tamaño de feature id es: ",features_id.shape)

    # Guardar las características de ese ID en un archivo
    features_id_path = os.path.join(output_dir, f"features_id_{id_value}.pkl")

    features_id.to_pickle(features_id_path)

    print(f"Características de ID={id_value} guardadas en: {features_id_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Procesando ID: 0...


Feature Extraction: 100%|██████████| 1/1 [00:26<00:00, 26.64s/it]


EL tamaño de feature id es:  (1, 777)
✅ Características de ID=0 guardadas en: /content/drive/MyDrive/features_blocks/features_id_0.pkl
Procesando ID: 1...


Feature Extraction: 100%|██████████| 1/1 [00:28<00:00, 28.35s/it]


EL tamaño de feature id es:  (1, 777)
✅ Características de ID=1 guardadas en: /content/drive/MyDrive/features_blocks/features_id_1.pkl
Procesando ID: 2...


...

EL tamaño de feature id es:  (1, 777)
✅ Características de ID=201 guardadas en: /content/drive/MyDrive/features_blocks/features_id_201.pkl
Procesando ID: 202...


Feature Extraction: 100%|██████████| 1/1 [00:04<00:00,  4.29s/it]


EL tamaño de feature id es:  (1, 777)
✅ Características de ID=202 guardadas en: /content/drive/MyDrive/features_blocks/features_id_202.pkl
Procesando ID: 203...


Feature Extraction: 100%|██████████| 1/1 [00:04<00:00,  4.89s/it]

EL tamaño de feature id es:  (1, 777)
✅ Características de ID=203 guardadas en: /content/drive/MyDrive/features_blocks/features_id_203.pkl


Procedemos a juntar todos los bloques

In [ ]:


# Ruta donde están guardados los archivos de características
input_dir = "/content/drive/MyDrive/features_blocks"
output_dir = "/content/drive/MyDrive/features_blocks/all_features"
os.makedirs(output_dir, exist_ok=True)
# Montamos Google Drive
drive.mount('/content/drive')
# Lista para almacenar cada DataFrame cargado
features_list = []

def extract_id_number(filename):
    """Se encarga de buscar el id del nombre del archivo para procesar de manera ordenada"""
    match = re.search(r"features_id_(\d+)\.pkl", filename)
    return int(match.group(1)) if match else -1

files = [f for f in os.listdir(input_dir) if f.startswith("features_id_") and f.endswith(".pkl")]
files_sorted = sorted(files, key=extract_id_number)

features_list = []
for file in files_sorted:
    print(f"Procesando archivo: {file}...")
    df = pd.read_pickle(os.path.join(input_dir, file))
    features_list.append(df)


# Concatenamos todos los DataFrames en uno solo
full_features_df = pd.concat(features_list, axis=0)

full_features_df.reset_index(drop=True, inplace=True)

full_features_df.to_pickle(os.path.join(output_dir, "all_features.pkl"))
# Mostrar información del DataFrame final
print("Shape del dataset final es: ", full_features_df.shape)
full_features_df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Procesando archivo: features_id_0.pkl...
Procesando archivo: features_id_1.pkl...
Procesando archivo: features_id_2.pkl...
Procesando archivo: features_id_3.pkl...
Procesando archivo: features_id_4.pkl...
Procesando archivo: features_id_5.pkl...
Procesando archivo: features_id_6.pkl...
Procesando archivo: features_id_7.pkl...
Procesando archivo: features_id_8.pkl...
Procesando archivo: features_id_9.pkl...
Procesando archivo: features_id_10.pkl...
Procesando archivo: features_id_11.pkl...
Procesando archivo: features_id_12.pkl...
Procesando archivo: features_id_13.pkl...
Procesando archivo: features_id_14.pkl...
Procesando archivo: features_id_15.pkl...
Procesando archivo: features_id_16.pkl...
Procesando archivo: features_id_17.pkl...
Procesando archivo: features_id_18.pkl...
Procesando archivo: features_id_19.pkl...
Procesando archivo: features_id_20.pkl...

,valor__variance_larger_than_standard_deviation,valor__has_duplicate_max,valor__has_duplicate_min,valor__has_duplicate,valor__sum_values,valor__abs_energy,valor__mean_abs_change,valor__mean_change,valor__mean_second_derivative_central,valor__median,...,valor__fourier_entropy__bins_5,valor__fourier_entropy__bins_10,valor__fourier_entropy__bins_100,valor__permutation_entropy__dimension_3__tau_1,valor__permutation_entropy__dimension_4__tau_1,valor__permutation_entropy__dimension_5__tau_1,valor__permutation_entropy__dimension_6__tau_1,valor__permutation_entropy__dimension_7__tau_1,valor__query_similarity_count__query_None__threshold_0.0,valor__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,0.0,1.0,-0.734864,33.127876,0.003897,-7.016291e-07,2.248299e-07,-0.000244,...,0.314393,0.418924,0.652842,1.222197,1.779523,2.375477,3.002084,3.596227,0.0,0.232910
1,0.0,0.0,0.0,1.0,0.300720,34.903172,0.004165,-5.418686e-07,-1.194152e-07,-0.001068,...,0.273489,0.370036,0.793862,1.148054,1.645622,2.196140,2.777400,3.365738,0.0,0.266963
2,0.0,0.0,0.0,1.0,-0.477569,16.164088,0.002264,-4.119189e-06,-1.837347e-06,-0.000092,...,0.235155,0.305728,0.574520,1.649418,2.545695,3.470808,4.431796,5.335461,0.0,0.207986
3,0.0,0.0,0.0,1.0,0.007569,6.087861,0.001689,-1.726074e-06,-1.227095e-07,0.000244,...,0.239211,0.339942,0.553027,1.663552,2.610122,3.569896,4.564650,5.506580,0.0,0.154306
4,0.0,0.0,0.0,1.0,-0.562194,28.034003,0.004050,1.992890e-07,-9.882629e-09,0.000809,...,0.367289,0.493589,0.808665,1.479723,2.184056,2.926665,3.690862,4.417537,0.0,0.336108


Repetimos el mismo procedimiento con la data de testeo